# [Module 1.1] 번역된 데이터 전처리

이 노트북은 아래와 같은 작업을 합니다.
- 번역된 데이터를 로딩 합니다.
- 리뷰에 워드 갯수를 추가하고, 10개 미만의 단어로 구성된 리뷰는 삭제 합니다.
- 특수문자 및 숫자를 리뷰에서 제거 합니다.
- 최종적으로 로컬에 파일을 저장 합니다.

## 번역된 데이터 로딩

In [1]:
%store -r full_translated_file_path

In [2]:
import pandas as pd
full_tr_df = pd.read_csv(full_translated_file_path)

In [3]:
full_tr_df.head(2)

,review_id,review_body_kr,review_body,product_category,star_rating,review_date
0,RIG9AWFOGRDVO,팝콘은 부패했다.,The popcorn was stale.,Grocery,2,2015-08-31
1,R1ODXB3C9UP3NL,괜찮은 제품으로 사용되었습니다.지금은 역겨운 소화가 어렵습니다.지난 몇 달 동안 C...,Used to be a decent product. Disgusting now a...,Grocery,1,2015-08-31


## 워드 개수 항목 추가 및 10개 미만 단어로 구성된 것 삭제

In [4]:
def add_word_cnt(full_tr_df):
    df = full_tr_df.copy()
    df.insert(1, 'kr_work_cnt', df.review_body_kr.astype(str).apply(lambda x: len(x.split(' '))))
    return df

word_df = add_word_cnt(full_tr_df)

In [5]:
word_df.kr_work_cnt.describe()

count    22009.000000
mean        23.372484
std         24.579677
min          1.000000
25%          7.000000
50%         15.000000
75%         30.000000
max        152.000000
Name: kr_work_cnt, dtype: float64

In [6]:
def clean_df(full_tr_df):
    df = full_tr_df[~full_tr_df.review_body_kr.isnull()]
    df = df[df.kr_work_cnt >= 10]
    df = df.drop(['review_body', 'product_category','star_rating','review_date'], axis=1)
    print("data shape:", df.shape)
    print("# of null: ", df.review_body_kr.isnull().sum())
    return df

cl_df = clean_df(word_df)

data shape: (14458, 3)
# of null:  0


In [7]:
pd.options.display.max_colwidth = 1000
pd.options.display.max_rows = 5
cl_df

,review_id,kr_work_cnt,review_body_kr
1,R1ODXB3C9UP3NL,32,괜찮은 제품으로 사용되었습니다.지금은 역겨운 소화가 어렵습니다.지난 몇 달 동안 Costco에서 구매한 3가지 모두 같은 결과로 끝납니다. 컨테이너를 열면 썩은 냄새가 납니다.그들이 지금이 분말을 만들고/처리하는 방법에 대해 옳지 않은 것.다시 사지 않을 것입니다.
2,R3T6TTD2IN0EFZ,118,"나는 변덕에 현지 슈퍼 마켓에서 이것을 구입하고 사람들에게 그것이 어떻게 맛인지 알리기로 결정했습니다.나는 땅콩 버터와 소금에 절인 카라멜의 거대한 팬입니다.<br /><br />예를 들어, 나는 오늘 소금에 절인 카라멜과 아몬드 종류 바를했다 그리고 그것은 놀라운했다.당신이 맛을 기대하는 것 같은 맛.<br /><br />그러나이 특정 제품은 화학 물질 유출과 같습니다.그것은 땅콩 버터 맛으로 시작하지만, 다음 (캐러멜 내가 추측 해요) 태우고 화학 같은 종류의 맛, 그리고 그것은 매우 짠 탄 맛으로 끝납니다.<br /><br />나는 숟가락에 몇 가지를 가지고 그것을 싫어, 그리고 나는 프레첼 빵에 몇 가지를 넣어 약간 맛이었다, 하지만 전반적으로 나는이 제품을 피할 것 소금에 절인 카라멜 땅콩 버터 같은 맛.<br /><br />그냥 재미를 위해, 나는 동료가 그것을 맛보고 그들 중 5 명 만이 제품이 “괜찮아”라고 생각했다.나머지 사람들이 제 정서를 공유했습니다."
...,...,...,...
22007,R1JWN8G9ED7LFC,11,이것들은 매우 부서지기 쉬운 상태입니다.그들은 부드럽게 다루지 않았고 좋은 맛도 없었습니다.
22008,R2X3CEOC27UTXP,13,이건 끔찍했어요너무 많은 통조림 커피 제품처럼 병약 한 달콤한.왜 커피처럼 맛볼 수 없을까요?


## 특수 문자 제거
병렬로 코어 수를 64로 지정했습니다. 특수 캐릭터(예: %, % " @ ) 및 숫자를 제거 합니다.

In [8]:
from utils import process_df_parallel, remove_special_chars_df
from functools import partial

# remove_special_chars_df(status_close_df, source_col='TITLE', target_col ='TITLE_CL')


df = process_df_parallel(cl_df, partial(remove_special_chars_df, 
                                     source_col='review_body_kr', target_col ='review_body_kr_cl'),
                         n_cores=64
                        )       



In [9]:
df

,review_id,kr_work_cnt,review_body_kr,review_body_kr_cl
1,R1ODXB3C9UP3NL,32,괜찮은 제품으로 사용되었습니다.지금은 역겨운 소화가 어렵습니다.지난 몇 달 동안 Costco에서 구매한 3가지 모두 같은 결과로 끝납니다. 컨테이너를 열면 썩은 냄새가 납니다.그들이 지금이 분말을 만들고/처리하는 방법에 대해 옳지 않은 것.다시 사지 않을 것입니다.,괜찮은 제품으로 사용되었습니다 지금은 역겨운 소화가 어렵습니다 지난 몇 달 동안 Costco에서 구매한 가지 모두 같은 결과로 끝납니다 컨테이너를 열면 썩은 냄새가 납니다 그들이 지금이 분말을 만들고 처리하는 방법에 대해 옳지 않은 것 다시 사지 않을 것입니다
2,R3T6TTD2IN0EFZ,118,"나는 변덕에 현지 슈퍼 마켓에서 이것을 구입하고 사람들에게 그것이 어떻게 맛인지 알리기로 결정했습니다.나는 땅콩 버터와 소금에 절인 카라멜의 거대한 팬입니다.<br /><br />예를 들어, 나는 오늘 소금에 절인 카라멜과 아몬드 종류 바를했다 그리고 그것은 놀라운했다.당신이 맛을 기대하는 것 같은 맛.<br /><br />그러나이 특정 제품은 화학 물질 유출과 같습니다.그것은 땅콩 버터 맛으로 시작하지만, 다음 (캐러멜 내가 추측 해요) 태우고 화학 같은 종류의 맛, 그리고 그것은 매우 짠 탄 맛으로 끝납니다.<br /><br />나는 숟가락에 몇 가지를 가지고 그것을 싫어, 그리고 나는 프레첼 빵에 몇 가지를 넣어 약간 맛이었다, 하지만 전반적으로 나는이 제품을 피할 것 소금에 절인 카라멜 땅콩 버터 같은 맛.<br /><br />그냥 재미를 위해, 나는 동료가 그것을 맛보고 그들 중 5 명 만이 제품이 “괜찮아”라고 생각했다.나머지 사람들이 제 정서를 공유했습니다.",나는 변덕에 현지 슈퍼 마켓에서 이것을 구입하고 사람들에게 그것이 어떻게 맛인지 알리기로 결정했습니다 나는 땅콩 버터와 소금에 절인 카라멜의 거대한 팬입니다 예를 들어 나는 오늘 소금에 절인 카라멜과 아몬드 종류 바를했다 그리고 그것은 놀라운했다 당신이 맛을 기대하는 것 같은 맛 그러나이 특정 제품은 화학 물질 유출과 같습니다 그것은 땅콩 버터 맛으로 시작하지만 다음 캐러멜 내가 추측 해요 태우고 화학 같은 종류의 맛 그리고 그것은 매우 짠 탄 맛으로 끝납니다 나는 숟가락에 몇 가지를 가지고 그것을 싫어 그리고 나는 프레첼 빵에 몇 가지를 넣어 약간 맛이었다 하지만 전반적으로 나는이 제품을 피할 것 소금에 절인 카라멜 땅콩 버터 같은 맛 그냥 재미를 위해 나는 동료가 그것을 맛보고 그들 중 명 만이 제품이 괜찮아 라고 생각했다 나머지 사람들이 제 정서를 공유했습니다
...,...,...,...,...
22007,R1JWN8G9ED7LFC,11,이것들은 매우 부서지기 쉬운 상태입니다.그들은 부드럽게 다루지 않았고 좋은 맛도 없었습니다.,이것들은 매우 부서지기 쉬운 상태입니다 그들은 부드럽게 다루지 않았고 좋은 맛도 없었습니다
22008,R2X3CEOC27UTXP,13,이건 끔찍했어요너무 많은 통조림 커피 제품처럼 병약 한 달콤한.왜 커피처럼 맛볼 수 없을까요?,이건 끔찍했어요너무 많은 통조림 커피 제품처럼 병약 한 달콤한 왜 커피처럼 맛볼 수 없을까요


### 데이터 Shuffling
데이터를 랜덤하게 혼합하여 다음 노트북에서 훈련에 사용 합니다.

In [10]:
def shuffle_data(df):
    shuffle_df = df.copy()
    shuffle_df = shuffle_df.sample(frac=1 , random_state=100)
    return shuffle_df

shuffle_df = shuffle_data(df)

In [11]:
shuffle_df.head()

,review_id,kr_work_cnt,review_body_kr,review_body_kr_cl
10970,R2QK1LUI0M7PCZ,33,"멋진 진저 맥주.지금까지, 내가 제일 좋아하는!나는 매우 적시에 발송물을 받았지만, 모든 외부 캔은 분쇄되어 플라스틱 안에 분사되었습니다.나는 소다 선적을위한 더 나은 내부 포장을 기대했다.적어도 소다와 상자 사이의 장벽.내 음료의 절반 이상이 패키지에 쏟아졌다.",멋진 진저 맥주 지금까지 내가 제일 좋아하는 나는 매우 적시에 발송물을 받았지만 모든 외부 캔은 분쇄되어 플라스틱 안에 분사되었습니다 나는 소다 선적을위한 더 나은 내부 포장을 기대했다 적어도 소다와 상자 사이의 장벽 내 음료의 절반 이상이 패키지에 쏟아졌다
21764,R38ZULX4B825J3,28,"리스의 “부활절 에그”는 초콜릿과 땅콩 버터의 녹은 풀로 도착했습니다.난 열지도 않았어썩은 운송 방법이 이 보물을 죽였는데, 이 보물이 제 아이들의 부활절 일요일을 밝히기 전에 말입니다.어찌됐어!크리스 트레이너 (뉴저지 주)",리스의 부활절 에그 는 초콜릿과 땅콩 버터의 녹은 풀로 도착했습니다 난 열지도 않았어썩은 운송 방법이 이 보물을 죽였는데 이 보물이 제 아이들의 부활절 일요일을 밝히기 전에 말입니다 어찌됐어 크리스 트레이너 뉴저지 주
16282,R3AVJYMU74ZCSP,25,밀가루 자체는 매우 좋습니다.내가 좋아하지 않는 것은 “최고”날짜가 3 개월 만에 있다는 것입니다.3 개월 만에 밀가루 4 봉지.우리 가정에서는 아니야그보다 더 긴 유통 기한이 있어야합니다.,밀가루 자체는 매우 좋습니다 내가 좋아하지 않는 것은 최고 날짜가 개월 만에 있다는 것입니다 개월 만에 밀가루 봉지 우리 가정에서는 아니야그보다 더 긴 유통 기한이 있어야합니다
17488,R3721QGZLV3TUC,22,한 번 진짜 코코넛으로 만든 좋은 제품 지금 농축에서 물로 희석!!이 따뜻한 마실 수 있기 전에 여전히 좋은 맛!더 이상 안돼!“원” 구매,한 번 진짜 코코넛으로 만든 좋은 제품 지금 농축에서 물로 희석 이 따뜻한 마실 수 있기 전에 여전히 좋은 맛 더 이상 안돼 원 구매
10074,R30QPJ5JJUAYM0,99,"나는 제품의 배달과 제품 자체에 실망했다.나는 그것을 추천 할 수 없다.제품이 녹은 플라스틱 얼음 팩으로 둘러싸여 도착했습니다.제품이 실온에서 상자에서 나오고 운송 중 높은 주변 온도를 통과했을 수 있습니다.제품을 사용하려고 할 때 거의 보이지 않는 내부 플라스틱 슬리브가 있음을 발견했습니다.그 결과, 나는 실수로 신선한 김이 나는 파스타 한 접시 위에 제품과 플라스틱 슬리브의 일부를 강판했다.플라스틱 조각을 발견했을 때 파스타를 버려야했습니다.마지막으로, 제품 표면에서 내부 플라스틱 슬리브를 떼어 낸 후, 나는 신선한 생선보다 통조림 고양이 사료에 더 연상시키는 제품에서 나오는 냄새를 감지했습니다.건강에 좋지 않은 결과없이 소량의 제품을 샘플링했지만 말린 생선 알보다 압축 된 소금처럼 맛이 더 좋다고 결론을 내렸습니다.나는 남아있는 것을 버렸다.",나는 제품의 배달과 제품 자체에 실망했다 나는 그것을 추천 할 수 없다 제품이 녹은 플라스틱 얼음 팩으로 둘러싸여 도착했습니다 제품이 실온에서 상자에서 나오고 운송 중 높은 주변 온도를 통과했을 수 있습니다 제품을 사용하려고 할 때 거의 보이지 않는 내부 플라스틱 슬리브가 있음을 발견했습니다 그 결과 나는 실수로 신선한 김이 나는 파스타 한 접시 위에 제품과 플라스틱 슬리브의 일부를 강판했다 플라스틱 조각을 발견했을 때 파스타를 버려야했습니다 마지막으로 제품 표면에서 내부 플라스틱 슬리브를 떼어 낸 후 나는 신선한 생선보다 통조림 고양이 사료에 더 연상시키는 제품에서 나오는 냄새를 감지했습니다 건강에 좋지 않은 결과없이 소량의 제품을 샘플링했지만 말린 생선 알보다 압축 된 소금처럼 맛이 더 좋다고 결론을 내렸습니다 나는 남아있는 것을 버렸다


## 전처리된 데이터를 로컬에 저장

In [12]:
import sagemaker
from utils import upload_s3
import os

bucket = sagemaker.Session().default_bucket()
prefix = 'ml-architect/topic'

data_folder = 'preproc_data/amazon_review'
os.makedirs(data_folder, exist_ok=True)



In [13]:
grocery_negative_path = os.path.join(data_folder, 'amazon_review_grocery_negative.csv')
shuffle_df.to_csv(grocery_negative_path, index=None)
# s3_amz_review_apparel_negative_path = upload_s3(bucket, local_negative_path, prefix)
# s3_amz_review_apparel_negative_path
%store grocery_negative_path

Stored 'grocery_negative_path' (str)
